In [18]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [19]:
def write_string_to_file(filename, content):
    lines = content.split('\n')
    cleaned_lines = [line.lstrip("=-").strip() for line in lines]
    cleaned_content = '\n'.join(cleaned_lines)
    try:
        with open(filename, 'w') as file:
            file.write(cleaned_content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [20]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Choose important cases to generate, important scenario is the scenario that users are more likely to encounter it.
- Limit the appearance of scenarios that are hard to happen. 
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""


FILTER_SYSTEM_PROMPT="""
Your task is to read all of this scenarios generation from many sources.
Then you remove all the same copies because there are many duplicate scenarios in meanings inside the input.
Make sure every scenarios in the response is unique.
Return scenario name only.
"""

# Identify what type of that element (button,buttons, icon,scroll, text field, radio buttons, menu, menu for navigation,menu for opening dialog or another menu, menu for filter, sliders, switches, dialog, link, form,rating, filter).


In [21]:
usecase_name = "RemoveAuthor"
project_name = "Online Library"
main_flow_prompt="""
Main scenario:
User clicks the "Delete" button (to the right from the author he/she wants to delete) on the author list/details page;
Application displays a dialog page to confirm destroying the record for chosen author;
User confirms deletion by pressing "Confirm" button.
"""

alt_prompt_1="""
Main scenario:
User clicks the "Delete" button (to the right from the author he/she wants to delete) on the author list/details page;
Application displays a dialog page to confirm destroying the record for chosen author;
User confirms deletion by pressing "Confirm" button.

Cancel operation scenario:
User clicks the "Delete" button (to the right from the author he/she wants to delete) on the author list/details page;
Application displays a dialog page to confirm destroying the record for chosen author;
User presses “Cancel” button;
Data doesn’t delete in the database, then a list of authors records is displayed to the user;
If the user selects the menu item "Books”, "Genres”, "Authors", "Home" or "API", the data will not be deleted from the database, and the corresponding form with updated data will be opened.
"""

prompt_all="""
Remove the author

Main scenario:
User clicks the "Delete" button (to the right from the author he/she wants to delete) on the author list/details page;
Application displays a dialog page to confirm destroying the record for chosen author;
User confirms deletion by pressing "Confirm" button.

Cancel operation scenario:
User clicks the "Delete" button (to the right from the author he/she wants to delete) on the author list/details page;
Application displays a dialog page to confirm destroying the record for chosen author;
User presses “Cancel” button;
Data doesn’t delete in the database, then a list of authors records is displayed to the user;
If the user selects the menu item "Books”, "Genres”, "Authors", "Home" or "API", the data will not be deleted from the database, and the corresponding form with updated data will be opened.
"""

In [22]:

promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

filtercontent = ""

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent =  "Conditional scenario:" + "\n" + condition_scenario_response


promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

filtercontent +="\n" + "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response
print(filtercontent)
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
write_string_to_file(usecase_name + "1.csv", filter_gpt_response)
print("\n" + "Final result:" + filter_gpt_response)

{}

Flow scenario:
1. Successful Author Deletion
2. User Cancels Deletion
3. Deletion Attempt of Non-Existent Author
4. Deletion Attempt with Insufficient Permissions
1. Cancel Deletion and Return to Author List
2. Cancel Deletion and Navigate to Books
3. Cancel Deletion and Navigate to Genres
4. Cancel Deletion and Navigate to Home
5. Cancel Deletion and Navigate to API
String has been written to RemoveAuthor1.csv

Final result:1. Successful Author Deletion
2. User Cancels Deletion
3. Deletion Attempt of Non-Existent Author
4. Deletion Attempt with Insufficient Permissions
5. Cancel Deletion and Return to Author List
6. Cancel Deletion and Navigate to Books
7. Cancel Deletion and Navigate to Genres
8. Cancel Deletion and Navigate to Home
9. Cancel Deletion and Navigate to API


In [23]:

promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

filtercontent = ""

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent =  "Conditional scenario:" + "\n" + condition_scenario_response


promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

filtercontent +="\n" + "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response
print(filtercontent)
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
write_string_to_file(usecase_name + "2.csv", filter_gpt_response)
print("\n" + "Final result:" + filter_gpt_response)

{}

Flow scenario:
1. Successful Author Deletion
2. User Cancels Deletion
3. Deletion Attempt of Non-Existent Author
4. Deletion Attempt with Insufficient Permissions
1. Cancel Deletion and Return to Author List
2. Cancel Deletion and Navigate to Books
3. Cancel Deletion and Navigate to Genres
4. Cancel Deletion and Navigate to Home
5. Cancel Deletion and Navigate to API
String has been written to RemoveAuthor2.csv

Final result:1. Successful Author Deletion
2. User Cancels Deletion
3. Deletion Attempt of Non-Existent Author
4. Deletion Attempt with Insufficient Permissions
5. Cancel Deletion and Return to Author List
6. Cancel Deletion and Navigate to Books
7. Cancel Deletion and Navigate to Genres
8. Cancel Deletion and Navigate to Home
9. Cancel Deletion and Navigate to API


In [24]:

promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

filtercontent = ""

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent =  "Conditional scenario:" + "\n" + condition_scenario_response


promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

filtercontent +="\n" + "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response
print(filtercontent)
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
write_string_to_file(usecase_name + "3.csv", filter_gpt_response)
print("\n" + "Final result:" + filter_gpt_response)

{}

Flow scenario:
Based on the provided main flow, here are the scenarios that can happen:

1. Successful Deletion Scenario: User successfully deletes the author record.
2. Cancellation Scenario: User decides not to delete the author record and cancels the operation.
3. Error Scenario: An error occurs preventing the deletion of the author record.
4. Network Failure Scenario: Network failure occurs after the user attempts to confirm deletion.
5. Session Timeout Scenario: User's session times out before confirming the deletion.
6. Multi-User Conflict Scenario: Another user modifies or deletes the author record before the user confirms deletion.
1. Cancel deletion via "Cancel" button
2. Cancel deletion and navigate to "Books"
3. Cancel deletion and navigate to "Genres"
4. Cancel deletion and navigate to "Authors"
5. Cancel deletion and navigate to "Home"
6. Cancel deletion and navigate to "API"
String has been written to RemoveAuthor3.csv

Final result:Successful Deletion Scenario
Cancel